# erg 文件处理

##  将指定文件夹下的所有 erg 文件合并为一个 csv

- 将 `path` 修改为 erg 文件所在的路径
  - 注意不要去掉示例中字符串 `r"D:\工作\2020\3月\UAES_C1.1_SPLIT\torque characteristic\Motor\400V" ` 前的 `r`

In [1]:
# ***
# 需要修改的参数
path = r"D:\工作\2020\3月\UAES_C1.1_SPLIT\torque characteristic\Motor\400V"   # 文件夹路径
# ***

from file_operator import FileOperator
import os

file_operator = FileOperator(path)
file_operator.handle_ergs(merge=True, file_name=os.path.join(path, path.split(os.path.sep)[-1] + '_merge_data.csv'))

##  将指定文件夹下的所有 erg 文件转换为单独的 csv

- 将 `path` 修改为 erg 文件所在的路径
  - 注意不要去掉示例中字符串 `r"D:\工作\2020\3月\UAES_C1.1_SPLIT\torque characteristic\Motor\400V" ` 前的 `r`

In [2]:
# ***
# 需要修改的参数
path = r"D:\工作\2020\3月\UAES_C1.1_SPLIT\eff\400V"   # 文件夹路径
# ***

from file_operator import FileOperator
import os

file_operator = FileOperator(path)
file_operator.handle_ergs(merge=False)

## 将指定的 erg 文件转换为 csv

- 将 file_name 修改为要转换的 erg 文件
  - 注意不要去掉示例中字符串 `r"D:\工作\2020\3月\UAES_C1.1_SPLIT\torque characteristic\Motor\400V\60s.erg" ` 前的 `r`

In [12]:
file_name = r"D:\工作\2020\3月\UAES_C1.1_SPLIT\torque characteristic\Motor\400V\60s.erg"   # erg 文件夹名称

from file_operator import FileOperator
import os

file_operator = FileOperator(file_name)
file_operator.handle_ergs(merge=False, file_name=''.join([os.path.splitext(file_name)[0], '.csv']))

# 按工况点求均值

- 目前只支持 csv 文件求取平均值
- file_name 参数
  - 如果为文件夹，则先会将此文件夹下的所有 csv 文件合并后，再按工况点求均值
  - 如果为文件，则只会对当前文件按工况点求均值
- `flag` 参数
  - 工况点标志位，需要根据实际情况修改
- 'handle_error_data'
  - 如果为 True， 则会按照 PA_signal < 1e10, 对数据进行筛选
  - 如果为 False， 则不会对数据进行筛选
  - 目的是为了防止出现假值

In [2]:
# ***
# 需要修改的参数

file_name = r"D:\工作\2020\3月\UAES_C1.1_SPLIT\eff\400V"
flag='speed_step'
handle_error_data = False
PA_signal = 'PA1_PM [W]'

# ***

from file_operator import FileOperator
import os

file_operator = FileOperator(file_name)
file_operator.get_average(flag=flag, handle_error_data=handle_error_data, PA_signal=PA_signal, file_name=None, head=True, line_count=None)

# 递归的处理文件夹中的所有 erg 文件并求平均值 

- 将 `path` 修改为 erg 文件所在的路径
  - 注意不要去掉示例中字符串 `r"D:\工作\2020\3月\UAES_C1.1_SPLIT" ` 前的 `r`

In [1]:
path = r"D:\工作\2020\3月\UAES_C1.1_SPLIT"

head = False  # 是否取前 n 个点
line_count = None  # 


import os
from file_operator import FileOperator

for root, dirs, _ in os.walk(path):
    for directory in dirs:
        sub_path = os.path.join(root, directory)
        _file_name =  "_".join(sub_path.split(os.path.sep)[len( path.split(os.path.sep) ):])
        result_dir = os.sep.join(path.split(os.sep)[:-1] + [path.split(os.sep)[-1] + 'result']
                + sub_path.split(os.path.sep)[len(path.split(os.path.sep) ):])
        file_operator = FileOperator(sub_path, result_dir)
        if file_operator.erg_files:
            file_name = os.path.join(result_dir,_file_name+".csv")
            file_operator.handle_ergs(merge=True, file_name=file_name)
            new_file_operator = FileOperator(result_dir)
            file_name = os.path.join(result_dir,"Average_"+_file_name+".csv")
            new_file_operator.get_average(file_name=file_name, head=head, line_count=line_count)

# 转换 DIAdem (.dat) 至 csv 文件 （测试）

- data_file 参数
  - 需要修改为 dat 文件，需要是文件，暂不支持目录
 - 由于 excel 会限制打开文件时显示的行数，因此当记录超过 $2^{20}$ 后，会分为多个 csv 文件进行储存，确保单个 csv 文件的行数不超过 $2^{20}$

In [1]:
# ***
# 需要修改的参数
dat_file = r"C:\Users\lijunjie3\Desktop\tmp\tmp\10℃\10℃-cycle3\measure 100hzdat base+uaes-c1.2-gdv-eef+data191226+time195708+n_cycle\measure 100hzdat base+uaes-c1.2-gdv-eef+data191226+time195708+n_cycle.dat"  # 换为自己的文件
# ***

import os
from collections import namedtuple, defaultdict
import numpy as np
import pandas as pd
import math


# 处理 data， 获取 head 信息
Channel = namedtuple('Channel', 'name, unit, channel_type,file_name, method, data_type, nums, start_index, block_offset')

with open(dat_file, encoding='cp1258') as f:
    text = f.read()
    

p = r"#BEGINCHANNELHEADER\n(.+?)#ENDCHANNELHEADER\n"
import re
channel_contents = re.findall(p, text, re.DOTALL)

if not channel_contents:
    with open(dat_file, encoding='utf16') as f:
        text = f.read()
        for line in f:
            print(line)
    channel_contents = re.findall(p, text, re.DOTALL)

channels = []
for channel_content in channel_contents:
    contents = channel_content.split('\n')
    dic = {}
    for content in contents:
        l = content.split(',')
        if len(l)>=2:
            dic[int(l[0])] = l[1]
    if dic.get(213) == 'BLOCK':
        channels.append(Channel(dic.get(200), dic.get(202), dic.get(210), dic.get(211), dic.get(213), dic.get(214), 
                                int(dic.get(220)),int(dic.get(221)), int(dic.get(222))))
    else:
        channels.append(Channel(dic.get(200), dic.get(202), dic.get(210), dic.get(211), dic.get(213), dic.get(214), int(dic.get(220)), int(dic.get(221)), None))



# 获取数据文件
datas = {}
datatypes = {'REAL32': 'float32', 'REAL64':'float64', 'INT16': 'int16', 'INT32':'int32'}
channel_datas = {}
for channel in channels:
    if channel.channel_type != 'EXPLICIT':
        raise Exception("Channel type {} is not supported".format(channel.channel_type))
        
    if channel.data_type not in datatypes:
        raise TypeError("Data Type {} is not supported in this version.".format(channel.data_type))
        
    if channel.file_name not in datas:
        datas[channel.file_name] = np.fromfile(os.path.join(os.path.split(dat_file)[0], channel.file_name), dtype=datatypes[channel.data_type])
        
    if channel.unit is not None:
        channel_name = "{} [{}]".format(channel.name, channel.unit)
    else:
        channel_name = channel.name
        
    if channel.method == 'CHANNEL':
        channel_datas[channel_name] = datas[channel.file_name][channel.start_index-1:channel.start_index + channel.nums - 1]
    elif channel.method == 'BLOCK':
        channel_datas[channel_name] = datas[channel.file_name][channel.start_index-1::channel.block_offset]
        

df = pd.DataFrame(channel_datas)
number_of_chunks = math.ceil(len(df) / 2** 20)
for id, df_i in enumerate(np.array_split(df, number_of_chunks)):
    df_i.to_csv('{}_{}.csv'.format(os.path.splitext(dat_file)[0], id+1), index=0, encoding='utf-8-sig')